# 8.2 Linear expressions

In [1]:
# install dependencies
%pip install -q amplpy

from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=['highs'],  # modules to install
    license_uuid='default',  # license to use
)  # instantiate AMPL object and register magics


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


An arithmetic expression is *linear* in a given variable if, for every unit increase or
decrease in the variable, the value of the expression increases or decreases by some fixed
amount. An expression that is linear in all its variables is called a *linear expression*.
(Strictly speaking, these are *affine* expressions, and a linear expression is an affine
expression with constant term zero. For simplicity, we will ignore this distinction.)

AMPL recognizes as a linear expression any sum of terms of the form

```text
constant-expr
variable-ref
(constant-expr) * variable-ref
```

provided that each `constant-expr` is an arithmetic expression that contains no variables,
while `variable-ref` is a reference (possibly subscripted) to a variable. The parentheses
around the `constant-expr` may be omitted if the result is the same according to the rules
of operator precedence (see [Table 7-3](../appendix/appendix.ipynb#table-7-3)).

The following examples, taken from the constraints in the multiperiod production model
of [Figure 6-3](../06/6_5_indexed_collections_of_sets.ipynb#fig-6-3), are all linear
expressions under this definition:

```ampl
avail[t]
Make[p,t] + Inv[p,t-1]
sum {p in PROD} (1 / rate[p]) * Make[p,t]
sum {a in AREA[p]} Sell[p,a,t] + Inv[p,t]
```

The model's objective,

```ampl
sum {p in PROD, t in 1..T}
  (sum {a in AREA[p]} revenue[p,a,t] * Sell[p,a,t]
   - prodcost[p] * Make[p,t] - invcost[p] * Inv[p,t])
```

is also linear, because subtraction of a term is the addition of its negative, and a sum of
sums is itself a sum.

Various kinds of expressions are equivalent to a sum of terms of the forms above, and are
also recognized as linear by AMPL. Division by an arithmetic expression is equivalent to
multiplication by its inverse, so

```ampl
(1 / rate[p]) * Make[p,t]
```

may be written in a linear program as

```ampl
Make[p,t] / rate[p]
```

The order of multiplications is irrelevant, so the `variable-ref` need not come at the end
of a term; for instance,

```ampl
revenue[p,a,t] * Sell[p,a,t]
```

is equivalent to

```ampl
Sell[p,a,t] * revenue[p,a,t]
```

As an example combining these principles, suppose that `revenue[p,a,t]` is measured in
dollars per metric ton, while `Sell[p,a,t]` remains in tons. If we define conversion factors

```ampl
param mt_t = 0.90718474;   # metric tons per ton
param t_mt = 1 / mt_t;    # tons per metric ton
```

then both

```ampl
sum {a in AREA[p]} mt_t * revenue[p,a,t] * Sell[p,a,t]
```

and

```ampl
sum {a in AREA[p]} revenue[p,a,t] * Sell[p,a,t] / t_mt
```

are linear expressions for total revenue.

To continue this example, if costs are also in dollars per metric ton, the objective could
be written as

```ampl
mt_t * sum {p in PROD, t in 1..T}
   (sum {a in AREA[p]} revenue[p,a,t] * Sell[p,a,t]
    - prodcost[p] * Make[p,t] - invcost[p] * Inv[p,t])
```

or as

```ampl
sum {p in PROD, t in 1..T}
   (sum {a in AREA[p]} revenue[p,a,t] * Sell[p,a,t]
    - prodcost[p] * Make[p,t] - invcost[p] * Inv[p,t]) / t_mt
```

Multiplication and division distribute over any summation to yield an equivalent linear
sum of terms. Notice that in the first form, `mt_t` multiplies the entire
`sum {p in PROD, t in 1..T}`, while in the second `t_mt` divides only the summand that
follows `sum {p in PROD, t in 1..T}`, because the `/` operator has higher precedence than
the `sum` operator. In these examples the effect is the same, however.

Finally, an `if–then–else` operator produces a linear result if the expressions following
`then` and `else` are both linear and no variables appear in the logical expression between
`if` and `else`. The following example appeared in a constraint in
[Section 7.3](../07/7_3_logical_and_conditional_expressions.ipynb):

```ampl
Make[j,t] +
   (if t = first(WEEKS) then inv0[j] else Inv[j,prev(t)])
```

The variables in a linear expression may not appear as operands to any other operators, or
as arguments to any functions. This rule applies to iterated operators such as `max`,
`min`, `abs`, `forall`, and `exists`, as well as to `^` and standard numerical functions
such as `sqrt`, `log`, and `cos`.

To summarize, a linear expression may be any sum of terms in the forms

```text
constant-expr
variable-ref
(constant-expr) * (linear-expr)
(linear-expr) * (constant-expr)
(linear-expr) / (constant-expr)
if logical-expr then linear-expr else linear-expr
```

where `constant-expr` is any arithmetic expression that contains no references to
variables, and `linear-expr` is any other (simpler) linear expression. Parentheses may be
omitted if the result is the same according to the rules of operator precedence in
[Table 7-3](../appendix/appendix.ipynb#table-7-3). AMPL automatically performs the
transformations that convert any such expression to a simple sum of linear terms.